# 气象数据数据处理

气象数据集说明文档
## 1.数据集信息
  数据集中文名称：中国地面降水日值0.5°×0.5°格点数据集（V2.0）
  数据集代码：SURF_CLI_CHN_PRE_DAY_GRID_0.5
  数据集版本：V2.0
  数据集建立时间：20120801
## 2.数据来源：
该数据集的数据来源包括2个部分：由国家气象信息中心基础资料专项收集、整理的1961年至最新的全国国家级台站（基本、基准和一般站）的降水日值资料；由GTOPO30数据（分辨率为0.05°×0.05°）经过重采样生产的中国陆地0.5°×0.5°的数字高程模型DEM。
## 3.数据集实体
###  3.1.数据集实体内容说明
#### 3.1.1.数据集实体文件名称：

中国降水日值0.5°×0.5°格点数据集文件命名由数据集代码（SURF_CLI_CHN_PRE_DAY_GRID_0.5）、年月日标识（YYYYMMDD）组成。
具体形式：SURF_CLI_CHN_PRE_DAY_GRID_0.5-YYYYMMDD.TXT
#### 3.1.2.数据集实体文件的内容描述：

数据集存储格式为ARCGIS标准格式，数据集实体包括1961年1月1日－至最新的逐日数据文件，每个文件中包括的前6行为头文件信息，其中：
第一行"ncols        128"表示实体数据有128列；
第二行"nrows       72"表示实体数据有72行；
第三行"xllcorner    72"表示数据最左下方格点单元的经度范围是72°-72.5°E；
第四行"yllcorner    18"表示数据最左下方格点单元的纬度范围是18°-18.5°N；
第五行"cellsize      0.5"表示网格是0.5°×0.5°的；
第六行"NODATA_value  -9999.0"表示中国区域以外的值用-9999.0表示。
从第七行开始是对应网格的降水值，第七行（降水数据第一行）第一列数据网格中心为（72.25°E ，53.75°N），第七行第二列数据网格中心为（72.75°E ，53.75°N），……，数据最后一行最后一列网格中心为（135.75°E ，18.25°N）。降水值保留1位小数。经度单位：度，纬度单位：度，格点降水单位：mm。
#### 3.1.3.特征值说明：
中国区域以外的值用-9999.0表示。


In [2]:
import numpy as np
import re
from glob import glob

import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed

In [9]:
txt_file = "/Users/huzheng/Downloads/气象数据集合/SURF_CLI_CHN_PRE_DAY_GRID_0.5_V2.0/1961/SURF_CLI_CHN_PRE_DAY_GRID_0.5-19610101.txt"


def txtfile2array(txtfile):
    with open(file=txtfile, mode='r', encoding='utf-8') as f:
        data = f.readlines()

    params = ['NCOLS', 'NROWS', 'XLLCORNER', 'YLLCORNER', 'CELLSIZE', 'NODATA_VALUE']
    params_dict = {}

    params_dict['date'] = re.findall(pattern="\-(.*?)\.txt", string=txtfile)[0]

    for index, temp_key in enumerate(params):
        params_dict[temp_key] = float([i for i in data[index].split(' ') if i != ''][1])

    out_array = np.concatenate(
        [np.array([float(i.strip()) for i in data[index].split(' ') if i != '']).reshape(1, -1) for index in
         range(len(params_dict) - 1, len(data))])
    return {'params': params_dict,
            'array': out_array}


txtfile2array(txtfile=txt_file)


{'params': {'date': '19610101',
  'NCOLS': 128.0,
  'NROWS': 72.0,
  'XLLCORNER': 72.0,
  'YLLCORNER': 18.0,
  'CELLSIZE': 0.5,
  'NODATA_VALUE': -9999.0},
 'array': array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]])}

In [11]:
base_dir = "/Users/huzheng/Downloads/气象数据集合/SURF_CLI_CHN_PRE_DAY_GRID_0.5_V2.0"
year_dir = glob(pathname=f"{base_dir}/*/*.txt")
year_dir = sorted(year_dir, key=lambda x: re.findall(pattern="\-(.*?)\.txt", string=x)[0])
year_dir[:5]

['/Users/huzheng/Downloads/气象数据集合/SURF_CLI_CHN_PRE_DAY_GRID_0.5_V2.0/1961/SURF_CLI_CHN_PRE_DAY_GRID_0.5-19610101.txt',
 '/Users/huzheng/Downloads/气象数据集合/SURF_CLI_CHN_PRE_DAY_GRID_0.5_V2.0/1961/SURF_CLI_CHN_PRE_DAY_GRID_0.5-19610102.txt',
 '/Users/huzheng/Downloads/气象数据集合/SURF_CLI_CHN_PRE_DAY_GRID_0.5_V2.0/1961/SURF_CLI_CHN_PRE_DAY_GRID_0.5-19610103.txt',
 '/Users/huzheng/Downloads/气象数据集合/SURF_CLI_CHN_PRE_DAY_GRID_0.5_V2.0/1961/SURF_CLI_CHN_PRE_DAY_GRID_0.5-19610104.txt',
 '/Users/huzheng/Downloads/气象数据集合/SURF_CLI_CHN_PRE_DAY_GRID_0.5_V2.0/1961/SURF_CLI_CHN_PRE_DAY_GRID_0.5-19610105.txt']

In [12]:
temp_result = Parallel(n_jobs=-1)(delayed(txtfile2array)(i) for i in tqdm(year_dir))

100%|██████████| 21197/21197 [00:17<00:00, 1183.01it/s]


In [13]:
final_alldata = np.concatenate([np.expand_dims(i['array'], axis=0) for i in temp_result], axis=0)
final_alldata.shape

(21197, 72, 128)

In [29]:
import pandas as pd
pd.to_datetime([i['params']['date'] for i in temp_result]).tolist()[:4]

[Timestamp('1961-01-01 00:00:00'),
 Timestamp('1961-01-02 00:00:00'),
 Timestamp('1961-01-03 00:00:00'),
 Timestamp('1961-01-04 00:00:00')]

## 更多操作

In [18]:
list(set([i['params']['NODATA_VALUE'] for i in temp_result]))[0]

-9999.0

In [19]:
data2 = final_alldata.copy()
data2[data2 == list(set([i['params']['NODATA_VALUE'] for i in temp_result]))[0]] = np.nan
data2

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan

135878739.4999997